### Import Packages

In [1]:
from nba_api.stats.static import teams
from nba_api.stats.static import players
from nba_api.stats.endpoints import leaguegamefinder

import datetime
import time
import random

import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
#Get team_ID to pull data // teamdashptpass for template
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamdashptpass
from nba_api.stats.endpoints import leaguegamefinder

### Set Up Team IDs

In [4]:
#Get entirely of List
allteams = teams.get_teams()

In [5]:
team_ids = []
team_names = []
for i in range(0, len(allteams)):
    team_ids.append(allteams[i]['id'])
    team_names.append(allteams[i]['nickname'])

In [6]:
allteams

[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Atlanta',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970},
 {'id': 1610612740,
  'full_name': 'New Orleans Pelicans',
  'abbreviation': 'NOP',
  'nickname': 'Pelicans',
  'city': 'New Orleans',
  'state': 'Louisiana',
  'year_founded': 2002},
 {'id': 1610612741,
  'full_name': 'Chicago Bulls',
  'abbreviation': 'CHI',
  'nickname': 'Bulls',
  'city': 'Chicago',
  'state': 'Illinois',
  'year_founded': 1966},
 {'id': 1610612742,
  'full_name': 'Dallas Mavericks',
  'abbreviation': 'DAL',
  'nickname': 'Mavericks',
  'city': 'Dallas',

In [19]:
game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable = 1610612759)
df = game_finder.get_data_frames()[0]
games_202122 = df[df.SEASON_ID.str[-4:] == '2021']
game_ids = games_202122[['TEAM_ID','TEAM_NAME','GAME_DATE','GAME_ID', 'WL', 'PLUS_MINUS']]

time.sleep(random.randrange(3, 5, 1))
#create columns for "Pass To" master df
df = teamdashptpass.TeamDashPtPass(team_id = 1610612759)
master_df_passto = df.get_data_frames()[0]
master_df_passto.drop(master_df_passto.index, inplace=True)
master_df_passto['GAME_DATE'] = ''
master_df_passto['WL'] = ''
master_df_passto['PLUS_MINUS'] = ''

time.sleep(random.randrange(2, 5, 1))
#the money code
for j in range(0,len(game_ids)):
        df = teamdashptpass.TeamDashPtPass(team_id = 1610612759, date_from_nullable=game_ids['GAME_DATE'][j], date_to_nullable=game_ids['GAME_DATE'][j])
        df_passto = df.get_data_frames()[0]
        df_passto['GAME_DATE'] = game_ids['GAME_DATE'][j]
        df_passto['WL'] = game_ids['WL'][j]
        df_passto['PLUS_MINUS'] = game_ids['PLUS_MINUS'][j]
        master_df_passto = pd.concat([master_df_passto,df_passto])

        time.sleep(random.randrange(3, 10, 1))

first_column = master_df_passto.pop('GAME_DATE')
master_df_passto.insert(0, 'GAME_DATE', first_column)
master_df_passto = master_df_passto.append(df_passto, ignore_index=True)
#master_df_passto.to_excel('datasets_EXPORT/Pass_Stats/' + team_names[i] + '_team_passto_stats.xlsx', index = False)

time.sleep(random.randrange(2, 5, 1))
WL_list = master_df_passto.groupby('GAME_DATE', as_index=False).agg({"PLUS_MINUS": "mean"})
passmade_df = master_df_passto.groupby('GAME_DATE', as_index=False).agg({"PASS": "sum"})
game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable = 1610612759)
df = game_finder.get_data_frames()[0]
games_202122 = df[df.SEASON_ID.str[-4:] == '2021']
TOV_list = games_202122[['GAME_DATE', 'AST' ,'TOV']]

merged_df = pd.merge(left=passmade_df, right=TOV_list, how='left', left_on='GAME_DATE', right_on='GAME_DATE')
merged_df['PASS/TO_RATIO'] = merged_df['PASS']/merged_df['TOV']
merged_df['AST/TO_RATIO'] = merged_df['AST']/merged_df['TOV']
merged_df['AST/PASS_RATIO'] = merged_df['AST']/merged_df['PASS']
merged_df = pd.merge(left=merged_df, right=WL_list, how='left', left_on='GAME_DATE', right_on='GAME_DATE')
merged_df['WL'] = ''
for k in range(0,len(merged_df)):
    if merged_df['PLUS_MINUS'][k] > 0:
        merged_df['WL'][k] = 'W'
    else:
        merged_df['WL'][k] = 'L'

merged_df['TEAM_NAME'] = team_names[i]
first_column = merged_df.pop('TEAM_NAME')
merged_df.insert(0, 'TEAM_NAME', first_column)


<ipython-input-19-062ef23f2bda>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['WL'][k] = 'W'
<ipython-input-19-062ef23f2bda>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['WL'][k] = 'L'


In [20]:
master_df_passto.to_excel('datasets_EXPORT/Pass_Stats/Spurs_team_passto_stats.xlsx', index = False)

In [21]:
game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable = 1610612758)
df = game_finder.get_data_frames()[0]
games_202122 = df[df.SEASON_ID.str[-4:] == '2021']
game_ids = games_202122[['TEAM_ID','TEAM_NAME','GAME_DATE','GAME_ID', 'WL', 'PLUS_MINUS']]

time.sleep(random.randrange(3, 5, 1))
#create columns for "Pass To" master df
df = teamdashptpass.TeamDashPtPass(team_id = 1610612758)
master_df_passto = df.get_data_frames()[0]
master_df_passto.drop(master_df_passto.index, inplace=True)
master_df_passto['GAME_DATE'] = ''
master_df_passto['WL'] = ''
master_df_passto['PLUS_MINUS'] = ''

time.sleep(random.randrange(2, 5, 1))
#the money code
for j in range(0,len(game_ids)):
        df = teamdashptpass.TeamDashPtPass(team_id = 1610612758, date_from_nullable=game_ids['GAME_DATE'][j], date_to_nullable=game_ids['GAME_DATE'][j])
        df_passto = df.get_data_frames()[0]
        df_passto['GAME_DATE'] = game_ids['GAME_DATE'][j]
        df_passto['WL'] = game_ids['WL'][j]
        df_passto['PLUS_MINUS'] = game_ids['PLUS_MINUS'][j]
        master_df_passto = pd.concat([master_df_passto,df_passto])

        time.sleep(random.randrange(3, 10, 1))

first_column = master_df_passto.pop('GAME_DATE')
master_df_passto.insert(0, 'GAME_DATE', first_column)
master_df_passto = master_df_passto.append(df_passto, ignore_index=True)
#master_df_passto.to_excel('datasets_EXPORT/Pass_Stats/' + team_names[i] + '_team_passto_stats.xlsx', index = False)

time.sleep(random.randrange(2, 5, 1))
WL_list = master_df_passto.groupby('GAME_DATE', as_index=False).agg({"PLUS_MINUS": "mean"})
passmade_df = master_df_passto.groupby('GAME_DATE', as_index=False).agg({"PASS": "sum"})
game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable = 1610612758)
df = game_finder.get_data_frames()[0]
games_202122 = df[df.SEASON_ID.str[-4:] == '2021']
TOV_list = games_202122[['GAME_DATE', 'AST' ,'TOV']]

merged_df = pd.merge(left=passmade_df, right=TOV_list, how='left', left_on='GAME_DATE', right_on='GAME_DATE')
merged_df['PASS/TO_RATIO'] = merged_df['PASS']/merged_df['TOV']
merged_df['AST/TO_RATIO'] = merged_df['AST']/merged_df['TOV']
merged_df['AST/PASS_RATIO'] = merged_df['AST']/merged_df['PASS']
merged_df = pd.merge(left=merged_df, right=WL_list, how='left', left_on='GAME_DATE', right_on='GAME_DATE')
merged_df['WL'] = ''
for k in range(0,len(merged_df)):
    if merged_df['PLUS_MINUS'][k] > 0:
        merged_df['WL'][k] = 'W'
    else:
        merged_df['WL'][k] = 'L'

merged_df['TEAM_NAME'] = team_names[i]
first_column = merged_df.pop('TEAM_NAME')
merged_df.insert(0, 'TEAM_NAME', first_column)


<ipython-input-21-0ccc98bcf316>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['WL'][k] = 'W'
<ipython-input-21-0ccc98bcf316>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['WL'][k] = 'L'


In [22]:
master_df_passto.to_excel('datasets_EXPORT/Pass_Stats/Kings_team_passto_stats.xlsx', index = False)

In [17]:
game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable = 1610612760)
df = game_finder.get_data_frames()[0]
games_202122 = df[df.SEASON_ID.str[-4:] == '2021']
game_ids = games_202122[['TEAM_ID','TEAM_NAME','GAME_DATE','GAME_ID', 'WL', 'PLUS_MINUS']]

time.sleep(random.randrange(3, 5, 1))
#create columns for "Pass To" master df
df = teamdashptpass.TeamDashPtPass(team_id = 1610612760)
master_df_passto = df.get_data_frames()[0]
master_df_passto.drop(master_df_passto.index, inplace=True)
master_df_passto['GAME_DATE'] = ''
master_df_passto['WL'] = ''
master_df_passto['PLUS_MINUS'] = ''

time.sleep(random.randrange(2, 5, 1))
#the money code
for j in range(0,len(game_ids)):
        df = teamdashptpass.TeamDashPtPass(team_id = 1610612760, date_from_nullable=game_ids['GAME_DATE'][j], date_to_nullable=game_ids['GAME_DATE'][j])
        df_passto = df.get_data_frames()[0]
        df_passto['GAME_DATE'] = game_ids['GAME_DATE'][j]
        df_passto['WL'] = game_ids['WL'][j]
        df_passto['PLUS_MINUS'] = game_ids['PLUS_MINUS'][j]
        master_df_passto = pd.concat([master_df_passto,df_passto])

        time.sleep(random.randrange(3, 10, 1))

first_column = master_df_passto.pop('GAME_DATE')
master_df_passto.insert(0, 'GAME_DATE', first_column)
master_df_passto = master_df_passto.append(df_passto, ignore_index=True)
#master_df_passto.to_excel('datasets_EXPORT/Pass_Stats/' + team_names[i] + '_team_passto_stats.xlsx', index = False)

time.sleep(random.randrange(2, 5, 1))
WL_list = master_df_passto.groupby('GAME_DATE', as_index=False).agg({"PLUS_MINUS": "mean"})
passmade_df = master_df_passto.groupby('GAME_DATE', as_index=False).agg({"PASS": "sum"})
game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable = 1610612760)
df = game_finder.get_data_frames()[0]
games_202122 = df[df.SEASON_ID.str[-4:] == '2021']
TOV_list = games_202122[['GAME_DATE', 'AST' ,'TOV']]

merged_df = pd.merge(left=passmade_df, right=TOV_list, how='left', left_on='GAME_DATE', right_on='GAME_DATE')
merged_df['PASS/TO_RATIO'] = merged_df['PASS']/merged_df['TOV']
merged_df['AST/TO_RATIO'] = merged_df['AST']/merged_df['TOV']
merged_df['AST/PASS_RATIO'] = merged_df['AST']/merged_df['PASS']
merged_df = pd.merge(left=merged_df, right=WL_list, how='left', left_on='GAME_DATE', right_on='GAME_DATE')
merged_df['WL'] = ''
for k in range(0,len(merged_df)):
    if merged_df['PLUS_MINUS'][k] > 0:
        merged_df['WL'][k] = 'W'
    else:
        merged_df['WL'][k] = 'L'

merged_df['TEAM_NAME'] = team_names[i]
first_column = merged_df.pop('TEAM_NAME')
merged_df.insert(0, 'TEAM_NAME', first_column)


<ipython-input-17-52d88792c7c3>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['WL'][k] = 'L'
<ipython-input-17-52d88792c7c3>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['WL'][k] = 'W'


In [18]:
master_df_passto.to_excel('datasets_EXPORT/Pass_Stats/Thunder_team_passto_stats.xlsx', index = False)

In [ ]:
passmade_df['TEAM_NAME'] = 'Detroit Pistons'

In [ ]:
passmade_df

In [ ]:
passmade_df.to_excel('datasets_EXPORT/Pass_Engineered_Stats/Hornets_PassRatio.xlsx', index = False)

### Set Up Required Files for Scrape

In [ ]:
for i in range(0, len(allteams)):
    game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable = team_ids[i])
    df = game_finder.get_data_frames()[0]
    games_202122 = df[df.SEASON_ID.str[-4:] == '2021']
    game_ids = games_202122[['TEAM_ID','TEAM_NAME','GAME_DATE','GAME_ID', 'WL', 'PLUS_MINUS']]

    time.sleep(random.randrange(3, 5, 1))
    #create columns for "Pass To" master df
    df = teamdashptpass.TeamDashPtPass(team_id = team_ids[i])
    master_df_passto = df.get_data_frames()[0]
    master_df_passto.drop(master_df_passto.index, inplace=True)
    master_df_passto['GAME_DATE'] = ''
    master_df_passto['WL'] = ''
    master_df_passto['PLUS_MINUS'] = ''

    time.sleep(random.randrange(3, 7, 1))
    #the money code
    for j in range(0,len(game_ids)):
            df = teamdashptpass.TeamDashPtPass(team_id = team_ids[i], date_from_nullable=game_ids['GAME_DATE'][j], date_to_nullable=game_ids['GAME_DATE'][j])
            df_passto = df.get_data_frames()[0]
            df_passto['GAME_DATE'] = game_ids['GAME_DATE'][j]
            df_passto['WL'] = game_ids['WL'][j]
            df_passto['PLUS_MINUS'] = game_ids['PLUS_MINUS'][j]
            master_df_passto = pd.concat([master_df_passto,df_passto])

            time.sleep(random.randrange(3, 10, 1))

    first_column = master_df_passto.pop('GAME_DATE')
    master_df_passto.insert(0, 'GAME_DATE', first_column)
    master_df_passto = master_df_passto.append(df_passto, ignore_index=True)
    master_df_passto.to_excel('datasets_EXPORT/Pass_StatsV2/' + team_names[i] + '_team_passto_stats.xlsx', index = False)
    
    time.sleep(random.randrange(3, 7, 1))
    WL_list = master_df_passto.groupby('GAME_DATE', as_index=False).agg({"PLUS_MINUS": "mean"})
    passmade_df = master_df_passto.groupby('GAME_DATE', as_index=False).agg({"PASS": "sum"})
    game_finder = leaguegamefinder.LeagueGameFinder(team_id_nullable = team_ids[i])
    df = game_finder.get_data_frames()[0]
    games_202122 = df[df.SEASON_ID.str[-4:] == '2021']
    TOV_list = games_202122[['GAME_DATE', 'AST' ,'TOV']]

    merged_df = pd.merge(left=passmade_df, right=TOV_list, how='left', left_on='GAME_DATE', right_on='GAME_DATE')
    merged_df['PASS/TO_RATIO'] = merged_df['PASS']/merged_df['TOV']
    merged_df['AST/TO_RATIO'] = merged_df['AST']/merged_df['TOV']
    merged_df['AST/PASS_RATIO'] = merged_df['AST']/merged_df['PASS']
    merged_df = pd.merge(left=merged_df, right=WL_list, how='left', left_on='GAME_DATE', right_on='GAME_DATE')
    merged_df['WL'] = ''
    for k in range(0,len(merged_df)):
        if merged_df['PLUS_MINUS'][k] > 0:
            merged_df['WL'][k] = 'W'
        else:
            merged_df['WL'][k] = 'L'

    merged_df['TEAM_NAME'] = team_names[i]
    first_column = merged_df.pop('TEAM_NAME')
    merged_df.insert(0, 'TEAM_NAME', first_column)

    merged_df.to_excel('datasets_EXPORT/Pass_Engineered_StatsV2/' + team_names[i] + '_PassRatio.xlsx', index = False)

    time.sleep(random.randrange(3, 7, 1))